In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset , DataLoader
import pandas as pd

Importing the Dataset

In [2]:
from sklearn.datasets import make_circles

In [3]:
make_circles

<function sklearn.datasets._samples_generator.make_circles(n_samples=100, *, shuffle=True, noise=None, random_state=None, factor=0.8)>

In [4]:
n_samples = 1000

Defining the input and output variable

In [5]:
X,y = make_circles(n_samples , noise = 0.03 , shuffle = True )

In [6]:
X

array([[ 0.49471828, -0.65781194],
       [ 0.88671537, -0.35065095],
       [ 0.05607902,  0.78284366],
       ...,
       [ 0.43195099,  0.72662444],
       [-0.84279774,  0.54948619],
       [ 0.45303983,  0.89235327]])

In [7]:
y

array([1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,

Preparing the DataFrame

In [8]:
circles_df = pd.DataFrame({"X1" : X[:,0] , "X2" : X[:,1] ,"label" : y})

In [9]:
circles_df.head(10)

,X1,X2,label
0,0.494718,-0.657812,1
1,0.886715,-0.350651,0
2,0.056079,0.782844,1
3,1.009525,0.175530,0
4,-0.231881,0.732019,1
5,0.332941,-0.762676,1
6,0.464530,0.640207,1
7,0.772744,-0.090455,1
8,-0.251660,0.979252,0
9,-0.702965,-0.678862,0


 Identifying the shape of the Dataset

In [10]:
print(f"The shape of input feature is {X.shape} , Shape of target feature is {y.shape}")

The shape of input feature is (1000, 2) , Shape of target feature is (1000,)


# 1 Turning our data to Tensors and creating the splits

In [11]:
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [12]:
X [:5] , y[:5]

(tensor([[ 0.4947, -0.6578],
         [ 0.8867, -0.3507],
         [ 0.0561,  0.7828],
         [ 1.0095,  0.1755],
         [-0.2319,  0.7320]]),
 tensor([1., 0., 1., 0., 1.]))

In [13]:
type(X) , X.dtype , type(y)

(torch.Tensor, torch.float32, torch.Tensor)

Splitting the data

In [14]:
from sklearn.model_selection import train_test_split
x_train , x_test = train_test_split(X , random_state = 42 , test_size = 0.2)
y_train , y_test = train_test_split(y , random_state = 42 , test_size = 0.2)

In [15]:
print(f"X_train's shape is {x_train.shape} , y_train's shape is {y_train.shape} , X_test's shape is {x_test.shape} , y_test's shape is {y_test.shape}")

X_train's shape is torch.Size([800, 2]) , y_train's shape is torch.Size([800]) , X_test's shape is torch.Size([200, 2]) , y_test's shape is torch.Size([200])


#2 Build or pick a pretrained model

To do so, we want to:
1. Setup device agonistic code so our code will run on an accelerator (GPU) if there is one


2. Construct a model (by subclassing nn.Module)


3. Define a loss function and optimizer


4. Create a training and test loop

In [16]:
#Make device agnostic code

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

Now we've setup device agnostic code, let's create a model that:
1. Subclasses nn. Module (almost all models in PyTorch subclass nn. Module)
2. Create 2 nn.Linear() layers that are capable of handling the shapes of our data
3. Defines a forward() method that outlines the forward pass (or forward computation) of the model
4. Instatiate an instance of our model class and send it to the target device

In [17]:
class CircleModel(nn.Module):
  #1.Construct a model that subclasses nn.Module
  def __init__(self):
    super().__init__()

    #2.Create 2 nn.Linear layers capable of handling the shapes of our data
    self.layer_1 = nn.Linear(in_features = 2 , out_features = 5)
    self.layer_2 = nn.Linear(in_features = 5 , out_features = 1)

    #3.Define a forward() method that outlines the forward pass
  def forward(self,x):
    return self.layer_2(self.layer_1(x)) #x -> layer_1 -> layer_2 -> output

#4.Instantiate instance of our model and send it to the target device
model = CircleModel()

In [18]:
model

CircleModel(
  (layer_1): Linear(in_features=2, out_features=5, bias=True)
  (layer_2): Linear(in_features=5, out_features=1, bias=True)
)

Random initialization of the model

In [19]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.4879,  0.4230],
         [ 0.2862, -0.0380],
         [-0.6275, -0.3430],
         [ 0.6686, -0.6493],
         [-0.5343,  0.6106]], requires_grad=True),
 Parameter containing:
 tensor([ 0.5299, -0.6079, -0.4656, -0.0793,  0.1479], requires_grad=True),
 Parameter containing:
 tensor([[ 0.1347,  0.0808,  0.4071, -0.1106, -0.1874]], requires_grad=True),
 Parameter containing:
 tensor([-0.1473], requires_grad=True)]

Replicating the above model using nn.Sequential()

This was the same as previous defined in class

In [20]:
model_0 = nn.Sequential(
   nn.Linear(in_features = 2 , out_features = 5),
   nn.Linear(in_features = 5 , out_features = 1)
)
model_0

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): Linear(in_features=5, out_features=1, bias=True)
)

In [21]:
model.state_dict()

OrderedDict([('layer_1.weight',
              tensor([[ 0.4879,  0.4230],
                      [ 0.2862, -0.0380],
                      [-0.6275, -0.3430],
                      [ 0.6686, -0.6493],
                      [-0.5343,  0.6106]])),
             ('layer_1.bias',
              tensor([ 0.5299, -0.6079, -0.4656, -0.0793,  0.1479])),
             ('layer_2.weight',
              tensor([[ 0.1347,  0.0808,  0.4071, -0.1106, -0.1874]])),
             ('layer_2.bias', tensor([-0.1473]))])

Setting up Loss function and optimizer

In [22]:
loss_fn = nn.BCEWithLogitsLoss() #sigmoid activation function

In [39]:
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters() , learning_rate)

Acuuracy

In [24]:
from sklearn.metrics import accuracy_score

##3.Train Model

To train our model , we're going to need to build a training loop

1.Forward pass

2.Calculate the Loss

3.Optimizer zero grad

4.Loss backward (Backpropagation)

5.Optimizer step (gradient descent)

In [25]:
model

CircleModel(
  (layer_1): Linear(in_features=2, out_features=5, bias=True)
  (layer_2): Linear(in_features=5, out_features=1, bias=True)
)

Our model outputs are going to be raw logits


We can convert these logits into prediction probabilities by passing them to some kind of activation function (eg. sigmoid for binary classification and softmax for multiclass classification)

Then we can convert our model's prediction probabilities to prediction labels by either rounding them or taking the argmax()

logits -> pred probs -> pred labels

In [26]:
model.eval()
with torch.inference_mode():
  y_logits = model(x_test)

In [27]:
y_logits[:5]

tensor([[-0.4748],
        [-0.3935],
        [-0.1655],
        [-0.4786],
        [-0.3020]])

In [28]:
y_test[:5]

tensor([0., 0., 0., 1., 0.])

In [29]:
#Use the sigmoid function on our model logits to turn them into prediction probabilities
y_pred_probs = torch.sigmoid(y_logits)
y_pred_probs[:5]

tensor([[0.3835],
        [0.4029],
        [0.4587],
        [0.3826],
        [0.4251]])

Now it's time to change it to either 0 or 1

In [30]:
y_pred = torch.round(y_pred_probs)[:5]

In [31]:
#Removing the extra dimensions
y_pred.squeeze()

tensor([0., 0., 0., 0., 0.])

In [32]:
y_pred

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]])

Building a training and testing loop

In [35]:
x_train[:5]

tensor([[-0.7149, -0.4308],
        [-0.9042, -0.4579],
        [ 0.0613, -1.0155],
        [-0.6722,  0.3750],
        [ 0.3262,  0.7146]])

In [37]:
y_train[:5]

tensor([1., 0., 0., 1., 1.])

In [42]:
from sklearn.metrics import accuracy_score

In [73]:
torch.manual_seed(42)
epochs = 101
for epoch in range(epochs):
  model.train()
  with torch.inference_mode():

    #1.Forward pass
    train_logits = model(x_train).squeeze()
    train_pred = torch.round(torch.sigmoid(y_logits))

    #2.Calculate the loss/accuarcy
    loss_train = loss_fn(train_logits,y_train)
    accuracy_train = accuracy_score(train_pred,y_train)*100
    loss_train.requires_grad_()

     #3.Optimizer.zero_grad
    optimizer.zero_grad()

     #4.Backpropagation
    loss_train.backward()

    #5.Optimizer.step()
    optimizer.step()


    model.eval()
  with torch.inference_mode():

    #1.Forward pass
    test_logits = model(x_test).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))

    #2.Calculate the loss/accuracy
    loss_test = loss_fn(test_logits,y_test)
    accuracy_test = accuracy_score(test_pred,y_test)*100

    #3.Optimizer.zero_grad
    optimizer.zero_grad()
    loss_test.requires_grad_()

    #4.Backpropagation
    loss_test.backward()

    #5.Optimizer.step
    optimizer.step()


    if(epoch%20==0):
      print(f"The epoch's iteration number is {epoch} \n \n Loss of train is {loss_train} \n accuracy of train is {accuracy_train}")
      print(f"Loss of test is {loss_test} \n accuracy of test is {accuracy_test}")
      print("\n \n")


The epoch's iteration number is 0 
 
 Loss of train is 0.7082985639572144 
 accuracy of train is 50.0
Loss of test is 0.7017697095870972 
 accuracy of test is 50.0

 

The epoch's iteration number is 20 
 
 Loss of train is 0.7082985639572144 
 accuracy of train is 50.0
Loss of test is 0.7017697095870972 
 accuracy of test is 50.0

 

The epoch's iteration number is 40 
 
 Loss of train is 0.7082985639572144 
 accuracy of train is 50.0
Loss of test is 0.7017697095870972 
 accuracy of test is 50.0

 

The epoch's iteration number is 60 
 
 Loss of train is 0.7082985639572144 
 accuracy of train is 50.0
Loss of test is 0.7017697095870972 
 accuracy of test is 50.0

 

The epoch's iteration number is 80 
 
 Loss of train is 0.7082985639572144 
 accuracy of train is 50.0
Loss of test is 0.7017697095870972 
 accuracy of test is 50.0

 

The epoch's iteration number is 100 
 
 Loss of train is 0.7082985639572144 
 accuracy of train is 50.0
Loss of test is 0.7017697095870972 
 accuracy of tes

#Improving a model

* Add more layers : Give the model more chances to learn about patterns in the data

* Add more hidden units : Go from 5 hidden units to 10 hidden units